# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/sg_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,56.62,56,69,24.47,RU,1691690522
1,1,invercargill,-46.4000,168.3500,33.03,92,100,3.00,NZ,1691690523
2,2,puerto penasco,31.3167,-113.5333,87.91,66,25,20.00,MX,1691690452
3,3,port-aux-francais,-49.3500,70.2167,41.00,93,100,33.02,TF,1691690532
4,4,puerto natales,-51.7236,-72.4875,38.97,92,100,6.89,CL,1691690538


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.7,
    color = "City"
)

# Display the map
map_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cdd = city_data_df
warm_cities_df = cdd.loc[(cdd["Max Temp"] > 90) & (cdd["Wind Speed"] < 10) & (cdd["Humidity"] > 30)]

# Drop any rows with null values
warm_cities_df = warm_cities_df.dropna(how="any")

# Display sample data
warm_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,massawa,15.6097,39.4500,99.73,37,28,3.49,ER,1691690565
89,89,brandon,27.9378,-82.2859,98.10,66,40,8.01,US,1691690398
108,108,sao felix do xingu,-6.6447,-51.9950,94.14,32,44,3.20,BR,1691690583
200,200,san buenaventura,27.0833,-101.5333,101.32,32,0,0.00,MX,1691690625
216,216,coahuayana de hidalgo,18.7000,-103.6583,91.83,54,99,4.54,MX,1691690631
220,220,edd,13.9297,41.6925,92.23,63,81,6.06,ER,1691690633
256,256,faya-largeau,17.9257,19.1043,96.84,37,7,7.31,TD,1691690651
263,263,mexicali,32.6519,-115.4683,99.09,35,0,5.01,MX,1691690537
292,292,lagunas,-5.2269,-75.6753,97.45,37,24,1.83,PE,1691690670
337,337,palmas,-10.2128,-48.3603,93.07,31,0,9.22,BR,1691690691


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = warm_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
64,massawa,ER,15.6097,39.4500,37,
89,brandon,US,27.9378,-82.2859,66,
108,sao felix do xingu,BR,-6.6447,-51.9950,32,
200,san buenaventura,MX,27.0833,-101.5333,32,
216,coahuayana de hidalgo,MX,18.7000,-103.6583,54,
220,edd,ER,13.9297,41.6925,63,
256,faya-largeau,TD,17.9257,19.1043,37,
263,mexicali,MX,32.6519,-115.4683,35,
292,lagunas,PE,-5.2269,-75.6753,37,
337,palmas,BR,-10.2128,-48.3603,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories, 
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    long = hotel_df.loc[index, "Lng"]
    lat = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
massawa - nearest hotel: Seghen International Hotel فندق دولي سيغن
brandon - nearest hotel: Brandon Motor Lodge
sao felix do xingu - nearest hotel: Hotel Terraço
san buenaventura - nearest hotel: Hotel Casa Bonita
coahuayana de hidalgo - nearest hotel: No hotel found
edd - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
mexicali - nearest hotel: No hotel found
lagunas - nearest hotel: Hospedaje Eco
palmas - nearest hotel: Hotel Triângulo Mineiro
carauari - nearest hotel: No hotel found
altar - nearest hotel: Hotel Altar Inn
bushehr - nearest hotel: هتل رستوران سیراف
tucurui - nearest hotel: No hotel found
san juan nepomuceno - nearest hotel: Hotel Imperial
kish - nearest hotel: هتل فارابی
san luis acatlan - nearest hotel: No hotel found
linden - nearest hotel: Watooka Guest House


,City,Country,Lat,Lng,Humidity,Hotel Name
64,massawa,ER,15.6097,39.4500,37,Seghen International Hotel فندق دولي سيغن
89,brandon,US,27.9378,-82.2859,66,Brandon Motor Lodge
108,sao felix do xingu,BR,-6.6447,-51.9950,32,Hotel Terraço
200,san buenaventura,MX,27.0833,-101.5333,32,Hotel Casa Bonita
216,coahuayana de hidalgo,MX,18.7000,-103.6583,54,No hotel found
220,edd,ER,13.9297,41.6925,63,No hotel found
256,faya-largeau,TD,17.9257,19.1043,37,No hotel found
263,mexicali,MX,32.6519,-115.4683,35,No hotel found
292,lagunas,PE,-5.2269,-75.6753,37,Hospedaje Eco
337,palmas,BR,-10.2128,-48.3603,31,Hotel Triângulo Mineiro


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.7,
    color = "Hotel Name", 
    hover_cols = ["Country", "Hotel Name"]
)


# Display the map
map_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)